In [5]:
import rdkit
from rdkit import Chem
import pandas as pd

In [7]:
df = pd.read_csv('MatDX_EF.csv')
df

,formula,space_group,structure,id,formation_energy
0,Ni4Ta6,R-3c,"[{'data': {'a': [-6.910143e-10, 0, 0], 'b': [3...",PN_MCKVEQNTJFRUHOURCR2HCEB35KDFRETL,{'reference': {'Ni': 'https://nomad-lab.eu/pro...
1,Mn4Sb2,I4_1/amd,"[{'data': {'a': [-1.87778212e-10, 1.87604249e-...",PN_FGGHB2NFWUCYD3YZO4FWN3QVOKCTVI6K,{'reference': {'Mn': 'https://nomad-lab.eu/pro...
2,Ir2Na2,Imma,"[{'data': {'a': [-1.36776653e-10, 2.60817299e-...",PN_BAO7ZSBZZCMA7PH7QMR7WLDI72GOELND,{'reference': {'Na': 'https://nomad-lab.eu/pro...
3,MoSm,R-3m,"[{'data': {'a': [1.65820251e-10, 9.57363439999...",PN_4YR2JLVZHJLC3F2TYPTOVDBJJ52C6XIE,{'reference': {'Mo': 'https://nomad-lab.eu/pro...
4,Sn17P12,I-43m,"[{'data': {'a': [-5.563178910000001e-10, 5.563...",PN_H6P36AVAKDU244QNQGGCJ7YLC4A4F2YE,{'reference': {'P': 'https://nomad-lab.eu/prod...
...,...,...,...,...,...
4995,Ag3Br,P4/mmm,"[{'data': {'a': [3.54083755e-10, 0, 0], 'b': [...",PN_GGP4J67ICWQHYNTSDBZQJFBJKFHKP2TY,{'reference': {'Br': 'https://nomad-lab.eu/pro...
4996,Al16Ge8,Fd-3m,"[{'data': {'a': [0, 6.01749067e-10, 6.01749067...",PN_HZ6TO4YLVA2K7PWWHZJRSFEAYOZLE3WF,{'reference': {'Al': 'https://nomad-lab.eu/pro...
4997,Pt2Th2,Cmcm,"[{'data': {'a': [-3.93770842e-10, 0, 0], 'b': ...",PN_5CPW22CGT4NOR6246BCGTZUO2UMDXXWI,{'reference': {'Th': 'https://nomad-lab.eu/pro...
4998,Ga5Sr,Cmmm,"[{'data': {'a': [2.2508728800000003e-10, -6.32...",PN_Y33JSPVMM6O4AYYYNR2LHUUT5WFVXJJC,{'reference': {'Ga': 'https://nomad-lab.eu/pro...
